In [3]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [4]:
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k = None)
classifier("I love this!")

Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764594301581383},
  {'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.00041385195800103247}]]

In [5]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [6]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.729603111743927},
  {'label': 'neutral', 'score': 0.1403854340314865},
  {'label': 'fear', 'score': 0.06816213577985764},
  {'label': 'joy', 'score': 0.04794234409928322},
  {'label': 'anger', 'score': 0.009156344458460808},
  {'label': 'disgust', 'score': 0.0026284705381840467},
  {'label': 'sadness', 'score': 0.0021221584174782038}],
 [{'label': 'neutral', 'score': 0.44937101006507874},
  {'label': 'disgust', 'score': 0.27359122037887573},
  {'label': 'joy', 'score': 0.10908293724060059},
  {'label': 'sadness', 'score': 0.09362731128931046},
  {'label': 'anger', 'score': 0.04047826677560806},
  {'label': 'surprise', 'score': 0.026970185339450836},
  {'label': 'fear', 'score': 0.006879061926156282}],
 [{'label': 'neutral', 'score': 0.6462167501449585},
  {'label': 'sadness', 'score': 0.24273262917995453},
  {'label': 'disgust', 'score': 0.04342261329293251},
  {'label': 'surprise', 'score': 0.028300542384386063},
  {'label': 'joy', 'score': 0.0142114

In [7]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [8]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [9]:
predictions[0]

[{'label': 'surprise', 'score': 0.729603111743927},
 {'label': 'neutral', 'score': 0.1403854340314865},
 {'label': 'fear', 'score': 0.06816213577985764},
 {'label': 'joy', 'score': 0.04794234409928322},
 {'label': 'anger', 'score': 0.009156344458460808},
 {'label': 'disgust', 'score': 0.0026284705381840467},
 {'label': 'sadness', 'score': 0.0021221584174782038}]

In [10]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [11]:
predictions[3]

[{'label': 'fear', 'score': 0.928167998790741},
 {'label': 'anger', 'score': 0.032190967351198196},
 {'label': 'neutral', 'score': 0.012808704748749733},
 {'label': 'sadness', 'score': 0.008756875991821289},
 {'label': 'surprise', 'score': 0.008597935549914837},
 {'label': 'disgust', 'score': 0.008431846275925636},
 {'label': 'joy', 'score': 0.0010455820010975003}]

In [12]:
predictions

[[{'label': 'surprise', 'score': 0.729603111743927},
  {'label': 'neutral', 'score': 0.1403854340314865},
  {'label': 'fear', 'score': 0.06816213577985764},
  {'label': 'joy', 'score': 0.04794234409928322},
  {'label': 'anger', 'score': 0.009156344458460808},
  {'label': 'disgust', 'score': 0.0026284705381840467},
  {'label': 'sadness', 'score': 0.0021221584174782038}],
 [{'label': 'neutral', 'score': 0.44937101006507874},
  {'label': 'disgust', 'score': 0.27359122037887573},
  {'label': 'joy', 'score': 0.10908293724060059},
  {'label': 'sadness', 'score': 0.09362731128931046},
  {'label': 'anger', 'score': 0.04047826677560806},
  {'label': 'surprise', 'score': 0.026970185339450836},
  {'label': 'fear', 'score': 0.006879061926156282}],
 [{'label': 'neutral', 'score': 0.6462167501449585},
  {'label': 'sadness', 'score': 0.24273262917995453},
  {'label': 'disgust', 'score': 0.04342261329293251},
  {'label': 'surprise', 'score': 0.028300542384386063},
  {'label': 'joy', 'score': 0.0142114

In [13]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156344458460808},
 {'label': 'disgust', 'score': 0.0026284705381840467},
 {'label': 'fear', 'score': 0.06816213577985764},
 {'label': 'joy', 'score': 0.04794234409928322},
 {'label': 'neutral', 'score': 0.1403854340314865},
 {'label': 'sadness', 'score': 0.0021221584174782038},
 {'label': 'surprise', 'score': 0.729603111743927}]

In [14]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [15]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [16]:
emotion_scores

{'anger': [0.06413368880748749,
  0.6126183867454529,
  0.06413368880748749,
  0.35148313641548157,
  0.08141235262155533,
  0.23222459852695465,
  0.5381835103034973,
  0.06413368880748749,
  0.3006700277328491,
  0.06413368880748749],
 'disgust': [0.27359122037887573,
  0.3482849597930908,
  0.10400675237178802,
  0.15072253346443176,
  0.18449512124061584,
  0.7271751165390015,
  0.15585488080978394,
  0.10400675237178802,
  0.27948105335235596,
  0.1779274195432663],
 'fear': [0.928167998790741,
  0.942527711391449,
  0.9723208546638489,
  0.36070719361305237,
  0.0950433686375618,
  0.051362842321395874,
  0.7474286556243896,
  0.40449562668800354,
  0.9155239462852478,
  0.051362842321395874],
 'joy': [0.9327970147132874,
  0.7044206857681274,
  0.76723712682724,
  0.2518812119960785,
  0.04056443274021149,
  0.043375901877880096,
  0.8725650906562805,
  0.04056443274021149,
  0.04056443274021149,
  0.04056443274021149],
 'sadness': [0.6462167501449585,
  0.8879395127296448,
  0.

In [17]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [00:54<00:00, 95.57it/s] 


In [18]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928168,0.932797,0.646217,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078766,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078766,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255169,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883199,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078766,9788185300535


In [19]:
books = pd.merge(books, emotions_df, on = "isbn13")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928168,0.932797,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704421,0.887940,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078766
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078766
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148209,0.030643,0.919165,0.255169,0.853721,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883199,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339218,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.064134,0.104007,0.459269,0.759455,0.951104,0.368111,0.078766


In [20]:
books.to_csv("books_with_emotions.csv", index = False)